<img style="float: center;" src='./assets/jWST-SS-1300x300_Banner2.jpg' width="1000px"/> 

In [ ]:
# Run this if you need it!
#! pip install ipympl --upgrade

In [ ]:
%matplotlib widget

In [ ]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from ipywidgets import interact, FloatSlider, IntSlider, Button, VBox, HBox

In [ ]:
image = fits.open('./epoch2_mosaic/f200w_epoch2_i2d.fits')[1].data

box_centers = []
with open('./diff/F200W_epoch2_epoch1.diff.stampxy.reg.all','r') as f:
    boxes = f.readlines()
for box in boxes:
    if box.startswith('image'):
        boxstr = box[box.find('(')+1:box.find(',11,')].split(',')
        if len(boxstr)==1:
            print(box)
        box_centers.append(np.array(boxstr).astype(int))


In [ ]:
box_size = 11  # pixels

# State variables
view_state = {"zoom": None}

plt.close()
view_state = {"zoom": None}
fig, ax = plt.subplots()

def plot_image():
    ax.clear()

    N = N_slider.value
    contrast = contrast_slider.value

    # Compute contrast bounds
    mean_val = np.nanmean(image)
    std_val = np.nanstd(image)
    vmin = mean_val - contrast * std_val
    vmax = mean_val + contrast * std_val

    if view_state["zoom"] is not None:
        (x0, x1, y0, y1) = view_state["zoom"]
        subimg = image[y0:y1, x0:x1]
        ax.imshow(subimg, origin='lower', cmap='gray', extent=(x0, x1, y0, y1),
                  vmin=vmin, vmax=vmax)
    else:
        ax.imshow(image, origin='lower', cmap='gray', vmin=vmin, vmax=vmax)
        # Draw grid
        ny, nx = image.shape
        for i in range(1, N):
            ax.axhline(i * ny / N, color='yellow', lw=0.5)
            ax.axvline(i * nx / N, color='yellow', lw=0.5)

    # Draw boxes
    half = box_size / 2
    for (xc, yc) in box_centers:
        rect = Rectangle((xc - half, yc - half), box_size, box_size,
                         linewidth=1, edgecolor='red', facecolor='none')
        ax.add_patch(rect)

    ax.set_title(f"Contrast: {contrast:.2f}")
    fig.canvas.draw_idle()

def on_click(event):
    if event.inaxes != ax:
        return
    if view_state["zoom"] is not None:
        return
    N = N_slider.value
    ny, nx = image.shape
    i = int(event.xdata / (nx / N))
    j = int(event.ydata / (ny / N))
    x0 = int(i * nx / N)
    x1 = int((i + 1) * nx / N)
    y0 = int(j * ny / N)
    y1 = int((j + 1) * ny / N)
    view_state["zoom"] = (x0, x1, y0, y1)
    plot_image()

def reset_view(_=None):
    view_state["zoom"] = None
    plot_image()

# Widgets
N_slider = IntSlider(value=5, min=0, max=25, step=1, description='Grid N')
contrast_slider = FloatSlider(value=1.0, min=0.1, max=3.0, step=0.1, description='Contrast')
reset_button = Button(description="Reset View", button_style='info')

# Link events (no lambdas — direct calls)
N_slider.observe(lambda change: plot_image(), names='value')
contrast_slider.observe(lambda change: plot_image(), names='value')
reset_button.on_click(reset_view)

# Connect click event
fig.canvas.mpl_connect('button_press_event', on_click)

# Display
controls = VBox([HBox([N_slider, contrast_slider]), reset_button])
display(controls)

# Initial draw
plot_image()